## Create Delta Silver Table

Transition data from a Bronze table to a Silver table in a structured and refined format. 

* Create the Silver Table
* Create a Working Table
* Handle Missing Values
* Adding Enriched data
* Insert Data into the Silver Table


In [0]:
%sql
-- Create Silver table and define a schema
CREATE OR REPLACE TABLE workspace.default.social_media_engagement_silver (
  post_id STRING,
  platform STRING,
  post_timestamp TIMESTAMP,
  likes DOUBLE,
  comments DOUBLE,
  shares DOUBLE,
  reach DOUBLE
);

-- Create a working table before inserting into the Silver table
CREATE OR REPLACE TEMP VIEW social_media_silver_working AS
  SELECT * FROM workspace.default.social_media_engagement_bronze;

SELECT * FROM social_media_silver_working;

In [0]:
%sql
-- Since the distribution in likes, shares, comments and reach
-- are normal Impute with the mean 
WITH mean_values AS (
  SELECT 
    AVG(likes) AS likes_mean,
    AVG(comments) AS comments_mean,
    AVG(shares) AS shares_mean,
    AVG(reach) AS reach_mean
  FROM social_media_silver_working
)

-- Update the table to replace null values with the calculated mean values
UPDATE social_media_silver_working
SET 
  likes = COALESCE(likes, (SELECT likes_mean FROM mean_values)),
  comments = COALESCE(comments, (SELECT comments_mean FROM mean_values)),
  shares = COALESCE(shares, (SELECT shares_mean FROM mean_values)),
  reach = COALESCE(reach, (SELECT reach_mean FROM mean_values));

In [0]:
%sql
-- Insert data from the source temp view (social_media_silver_working)
-- into the target table (social_media_engagement_silver)
INSERT INTO workspace.default.social_media_engagement_silver 
SELECT 
  post_id, 
  platform,
  post_timestamp, 
  likes, 
  shares, 
  comments, 
  reach
FROM social_media_silver_working